In [33]:
import csv
from urllib.request import urlopen
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from statistics import mean
import sys

In [41]:
def getSoupFromURL(url, method='get', data={}):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

In [35]:
models = ['INTENS', 'LIFE', 'ZEN']
years = ['2012', '2013', '2014', '2015', '2016', '2017']
da = pd.DataFrame(index =models, columns=years)
def getArgus(models, years):
    for model in models:
        for year in years:
            url='https://www.lacentrale.fr/cote-auto-renault-zoe-' + model + '+charge+rapide-' + year + '.html'
            r = urlopen(url).read()
            price = int(re.findall(b': (\d+\s\d+)\s&euro',r)[0].decode("utf-8").replace(" ", ""))
            da.loc[model][year]=price
    return da
            
da = getArgus(models, years)

In [49]:
def getURLs(region):
    urls = ['https://www.leboncoin.fr/voitures/offres/' + region + '/?th=1&q=Renault%20Zo%E9']
    for i in range(2,15):
        url = 'https://www.leboncoin.fr/voitures/offres/' + region + '/?o=' + str(i) + '&q=Renault%20Zo%E9'
        soup = getSoupFromURL(url)
        if soup.find_all('article', class_="noResult clearfix listing_thumbs ab-testing_A") != []:
            break
        urls.append(url)
    return urls

['https://www.leboncoin.fr/voitures/offres/ile_de_france/?th=1&q=Renault%20Zo%E9', 'https://www.leboncoin.fr/voitures/offres/ile_de_france/?o=2&q=Renault%20Zo%E9', 'https://www.leboncoin.fr/voitures/offres/ile_de_france/?o=3&q=Renault%20Zo%E9']


In [51]:
page='https://www.leboncoin.fr/voitures/offres/ile_de_france/?th=1&q=Renault%20Zo%E9'

def getLinks(urls):
    for url in urls:
        soup = getSoupFromURL(url)    
        cars = soup.find_all("a", class_="list_item clearfix trackable")
        links = ["https:" + car['href'] for car in cars]

    return links
print(getLinks(getURLs('ile_de_france')))

['https://www.leboncoin.fr/voitures/1136394587.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1148864542.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1306769563.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1147612495.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1101078242.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1167828367.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1171143468.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1303806454.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1178596024.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1303298657.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1166590040.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1303298374.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1166018156.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1166018166.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1187231493.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1187231658.htm?ca=12_s', 'https://www.leboncoin.fr/voitures/1120

In [52]:
def getData(links):
    annee_l = []
    prix_l = []
    km_l = []
    type_vendeur_l = []
    version_l = []
    argus_l = []
    for link in links:

        l = urlopen(link).read()
        annee = int(re.findall(b'annee\s:\s\"(\d{4})', l)[0].decode("utf-8"))
        annee_l.append(annee)

        prix = int(re.findall(b'prix\s:\s\"(\d+)', l)[0].decode("utf-8"))
        prix_l.append(prix)
        
        k=re.findall(b'(\d+?\s\d+)\sKM', l)
        if len(k)!=0:
            km = k[0].decode("utf-8")
        else:
            km = "N/A"
        km_l.append(km)

        tv = re.findall(b'Pro V', l)
        if len(tv) != 0:
            type_vendeur = 'Professionnel'
        else:
            type_vendeur = 'Particulier'
        type_vendeur_l.append(type_vendeur)

        v=re.findall(b'zen(?i)|life(?i)|intens(?i)', l)
        if len(v)!= 0:
            version = v[0].decode("utf-8").upper()
        else:
            version = "N/A"
        version_l.append(version)
        
        if version != "N/A":
            argus = int(da.loc[version][str(annee)])
        else:
            argus='N/A'
        argus_l.append(argus)

    df = pd.DataFrame({'Annonce': links, 'Année': annee_l,
                       'Prix': prix_l, 'Vendeur': type_vendeur_l, 'Version': version_l, 'Argus':argus_l})
    return df


df = getData(getLinks(getURLs('ile_de_france')))
print(df.head())

                                             Annonce  Année  Argus   Prix  \
0  https://www.leboncoin.fr/voitures/1136394587.h...   2014  10212   9600   
1  https://www.leboncoin.fr/voitures/1148864542.h...   2014   9908  10990   
2  https://www.leboncoin.fr/voitures/1306769563.h...   2017  13984  15000   
3  https://www.leboncoin.fr/voitures/1147612495.h...   2015  11405   9500   
4  https://www.leboncoin.fr/voitures/1101078242.h...   2013   9335   9490   

         Vendeur Version  
0  Professionnel    LIFE  
1  Professionnel  INTENS  
2    Particulier     ZEN  
3  Professionnel    LIFE  
4  Professionnel    LIFE  


In [75]:
df["Comparaison"] = df.loc[df['Argus']!='N/A'].eval("Prix > Argus")
df['Comparaison'] = df["Comparaison"].replace(False, "Moins").replace(True, "Plus")

In [76]:
df.head()

,Annonce,Année,Argus,Prix,Vendeur,Version,Comparaison
0,https://www.leboncoin.fr/voitures/1321487407.h...,2017,N/A,15900,Particulier,N/A,NaN
1,https://www.leboncoin.fr/voitures/1317781865.h...,2015,12436,10990,Professionnel,INTENS,Moins
2,https://www.leboncoin.fr/voitures/1321415628.h...,2014,10212,7900,Professionnel,LIFE,Moins
3,https://www.leboncoin.fr/voitures/1317567777.h...,2013,9335,8990,Professionnel,LIFE,Moins
4,https://www.leboncoin.fr/voitures/1321142626.h...,2014,10212,9700,Particulier,LIFE,Moins


In [77]:
def getDataRegion(region):
    urls = getLinks(getURLs(region))
    df = getData(urls)
    if 'N/A' in df['Argus']:
        df["Comparaison"] = df.loc[df['Argus']!='N/A'].eval("Prix > Argus")
    else:
        df["Comparaison"] = df.eval("Prix > Argus")
    df['Comparaison'] = df["Comparaison"].replace(False, "Moins").replace(True, "Plus")
    df.insert(loc=0, column='Région', value=region)
    return df
    

In [75]:
df_idf = getDataRegion('ile_de_france')

In [78]:
df_paca = getDataRegion('provence_alpes_cote_d_azur')

In [79]:
df_aquitaine = getDataRegion('aquitaine')

In [80]:
df = pd.concat([df_idf, df_paca, df_aquitaine])

In [83]:
df.to_csv('exo_dom_lesson_4.csv')